<a href="https://colab.research.google.com/github/DmitriiKnyazkin/Yolo_TEST/blob/master/MP_ver_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mediapipe
!wget -O face_landmarker_v2_with_blendshapes.task -q https://storage.googleapis.com/mediapipe-models/face_landmarker/face_landmarker/float16/1/face_landmarker.task


In [2]:
import cv2
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
from google.colab.patches import cv2_imshow

In [5]:
base_options = python.BaseOptions(model_asset_path='face_landmarker_v2_with_blendshapes.task')
options = vision.FaceLandmarkerOptions(base_options=base_options,
                                       output_face_blendshapes=True,
                                       output_facial_transformation_matrixes=True,
                                       num_faces=1)
detector = vision.FaceLandmarker.create_from_options(options)

In [6]:
cap = cv2.VideoCapture('/content/video_in.mp4')
imageWidth = cap.get(3)# Разрешение входного файла, ширина
imageHeight = cap.get(4)# Разрешение входного файла, высота
fps = cap.get(cv2.CAP_PROP_FPS)  # Частота входного файла, FPS
len_freame = int(cap.get(7)+1) # Длина выходного файла, кадры
output_width = 640 # Разрешение выходного файла, ширина
output_height = 480 # Разрешение выходного файла, высота

writer = cv2.VideoWriter("/content/video_out.avi", cv2.VideoWriter_fourcc(*"XVID"), fps, (output_width, output_height))

for i in range(len_freame):
  cap.set(cv2.CAP_PROP_POS_FRAMES, i-1)
  res, frame = cap.read()
  mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
  detection_result = detector.detect(mp_image)
  detection_result.face_landmarks[0][6].y
  detection_result.face_landmarks[0][6].x
  #Высчитываем координаты кадра относительно ценра глаз (точка 6)
  y_zenter = detection_result.face_landmarks[0][6].y * imageHeight
  x_zenter = detection_result.face_landmarks[0][6].x * imageWidth
  y_0 = int(y_zenter - output_height/2)
  y_1 = y_0 + output_height
  x_0 = int(x_zenter - output_width/2)
  x_1 = x_0 + output_width
  frame_cut_out = frame[y_0:y_1, x_0:x_1]
  writer.write(frame_cut_out)

writer.release()